## Deliverable 2. Create a Customer Travel Destinations Map.

In [6]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [7]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Noormarkku,FI,3/24/2021 20:03,61.5927,21.8685,41.00,75,0,8.05,clear sky
1,1,Khatanga,RU,3/24/2021 20:03,71.9667,102.5000,-3.10,93,100,3.11,overcast clouds
2,2,Alyangula,AU,3/24/2021 20:03,-13.8483,136.4192,81.19,83,100,6.96,overcast clouds
3,3,Kapaa,US,3/24/2021 20:03,22.0752,-159.3190,78.01,69,40,8.05,scattered clouds
4,4,Bluff,NZ,3/24/2021 20:00,-46.6000,168.3333,46.99,97,14,8.23,few clouds


In [8]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for you trip?"))

What is the minimum temperature you would like for your trip?30
What is the maximum temperature you would like for you trip?40


In [9]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &(city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
8,8,Kyzyl-Suu,KG,3/24/2021 20:03,42.3414,78.0056,39.09,42,97,9.82,overcast clouds
10,10,Nakhodka,RU,3/24/2021 20:03,42.8333,132.8947,37.08,76,100,5.41,overcast clouds
57,57,Honningsvag,NO,3/24/2021 20:03,70.9821,25.9704,39.20,65,0,12.66,clear sky
60,60,Kovdor,RU,3/24/2021 20:03,67.5662,30.4758,34.27,83,79,16.69,broken clouds
83,83,Olafsvik,IS,3/24/2021 20:03,64.8945,-23.7142,33.60,62,96,16.69,overcast clouds


In [10]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().count()

City_ID                69
City                   69
Country                69
Date                   69
Lat                    69
Lng                    69
Max Temp               69
Humidity               69
Cloudiness             69
Wind Speed             69
Weather Description    69
dtype: int64

In [11]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Date                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Weather Description    0
dtype: int64

In [14]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
8,Kyzyl-Suu,KG,39.09,overcast clouds,42.3414,78.0056,
10,Nakhodka,RU,37.08,overcast clouds,42.8333,132.8947,
57,Honningsvag,NO,39.20,clear sky,70.9821,25.9704,
60,Kovdor,RU,34.27,broken clouds,67.5662,30.4758,
83,Olafsvik,IS,33.60,overcast clouds,64.8945,-23.7142,
90,Kautokeino,NO,30.09,overcast clouds,69.0125,23.0412,
119,Homer,US,36.00,broken clouds,59.6425,-151.5483,
120,Vardo,NO,33.80,overcast clouds,70.3705,31.1107,
127,Ostrovnoy,RU,35.71,overcast clouds,68.0531,39.5131,
130,Berlevag,NO,35.60,clear sky,70.8578,29.0864,


In [15]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...........")
        

Hotel not found...........
Hotel not found...........
Hotel not found...........
Hotel not found...........
Hotel not found...........
Hotel not found...........
Hotel not found...........
Hotel not found...........


In [16]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [17]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [20]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Weather Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [21]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.75)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))